In [1]:
#this program will only work zumi python library 1.2 and above
from zumi.zumi import Zumi
import time
import IPython.display
import numpy as np
import math 

def data_regularize(data, type="spherical", divs=10):
    limits = np.array([
        [min(data[:, 0]), max(data[:, 0])],
        [min(data[:, 1]), max(data[:, 1])],
        [min(data[:, 2]), max(data[:, 2])]])
        
    regularized = []

    if type == "cubic": # take mean from points in the cube
        
        X = np.linspace(*limits[0], num=divs)
        Y = np.linspace(*limits[1], num=divs)
        Z = np.linspace(*limits[2], num=divs)

        for i in range(divs-1):
            for j in range(divs-1):
                for k in range(divs-1):
                    points_in_sector = []
                    for point in data:
                        if (point[0] >= X[i] and point[0] < X[i+1] and
                                point[1] >= Y[j] and point[1] < Y[j+1] and
                                point[2] >= Z[k] and point[2] < Z[k+1]):
                            points_in_sector.append(point)
                    if len(points_in_sector) > 0:
                        regularized.append(np.mean(np.array(points_in_sector), axis=0))

    elif type == "spherical": #take mean from points in the sector
        divs_u = divs 
        divs_v = divs * 2

        center = np.array([
            0.5 * (limits[0, 0] + limits[0, 1]),
            0.5 * (limits[1, 0] + limits[1, 1]),
            0.5 * (limits[2, 0] + limits[2, 1])])
        d_c = data - center
    
        #spherical coordinates around center
        r_s = np.sqrt(d_c[:, 0]**2. + d_c[:, 1]**2. + d_c[:, 2]**2.)
        d_s = np.array([
            r_s,
            np.arccos(d_c[:, 2] / r_s),
            np.arctan2(d_c[:, 1], d_c[:, 0])]).T

        u = np.linspace(0, np.pi, num=divs_u)
        v = np.linspace(-np.pi, np.pi, num=divs_v)

        for i in range(divs_u - 1):
            for j in range(divs_v - 1):
                points_in_sector = []
                for k, point in enumerate(d_s):
                    if (point[1] >= u[i] and point[1] < u[i + 1] and
                            point[2] >= v[j] and point[2] < v[j + 1]):
                        points_in_sector.append(data[k])

                if len(points_in_sector) > 0:
                    regularized.append(np.mean(np.array(points_in_sector), axis=0))

    return np.array(regularized)


# http://www.mathworks.com/matlabcentral/fileexchange/24693-ellipsoid-fit
# for arbitrary axes
def ellipsoid_fit(X):
    x = X[:, 0]
    y = X[:, 1]
    z = X[:, 2]
    D = np.array([x * x + y * y - 2 * z * z,
                 x * x + z * z - 2 * y * y,
                 2 * x * y,
                 2 * x * z,
                 2 * y * z,
                 2 * x,
                 2 * y,
                 2 * z,
                 1 - 0 * x])
    d2 = np.array(x * x + y * y + z * z).T # rhs for LLSQ
    u = np.linalg.solve(D.dot(D.T), D.dot(d2))
    a = np.array([u[0] + 1 * u[1] - 1])
    b = np.array([u[0] - 2 * u[1] - 1])
    c = np.array([u[1] - 2 * u[0] - 1])
    v = np.concatenate([a, b, c, u[2:]], axis=0).flatten()
    A = np.array([[v[0], v[3], v[4], v[6]],
                  [v[3], v[1], v[5], v[7]],
                  [v[4], v[5], v[2], v[8]],
                  [v[6], v[7], v[8], v[9]]])

    center = np.linalg.solve(- A[:3, :3], v[6:9])
    
    translation_matrix = np.eye(4)
    translation_matrix[3, :3] = center.T

    R = translation_matrix.dot(A).dot(translation_matrix.T)

    evals, evecs = np.linalg.eig(R[:3, :3] / -R[3, 3])
    evecs = evecs.T

    radii = np.sqrt(1. / np.abs(evals))
    radii *= np.sign(evals)

    return center, evecs, radii, v

def compass_auto_calibration(iterations=1000):
    print("Starting auto-calibration")
    print("rotate Zumi around in all directions")
    print("make sure to avoid large metal")
    print("objects and magnetic materials.")
    time.sleep(2)
    data_list = []
    for i in range(iterations):
        data = zumi.compass.get_magnet_raw()
        data_list.append(data)
        print(data)
        IPython.display.clear_output(wait=True) 
        
    print("done taking samples")
    print("saving data")
    time.sleep(2)
    file = open("mag_out.txt", "w")
    for val in data_list:
        file.write( str(val[0])+" "
                   +str(val[1])+" "
                   +str(val[2])+"\n")
    file.close()
    print("done saving data")
    time.sleep(1)
    #try to do a best fit
    try:
        data = np.loadtxt("mag_out.txt")
        center, evecs, radii, v = ellipsoid_fit(data)
        a, b, c = radii
        r = (a * b * c) ** (1.0 / 3.0)
        D = np.array([[r/a, 0., 0.], [0., r/b, 0.], [0., 0., r/c]])
        transformation = evecs.dot(D).dot(evecs.T)
        print('center: ', center)
        print('radii: ', radii)
        np.savetxt('magcal_ellipsoid.txt', np.vstack((center.T, transformation)))
    finally:
        pass
    print("Done with auto-calibration")
    return center, radii, transformation, evecs, v

In [4]:
zumi = Zumi()
zumi.reset_gyro()
zumi.MIN_I2C_DELAY = 0.01

center, radii, transformation, evecs, v = compass_auto_calibration(500)

done taking samples
saving data
done saving data
center:  [-1498.95380107  1642.24486383  4885.22294537]
radii:  [1566.19676204 2055.12161137 1869.01984246]
Done with auto-calibration


In [3]:
import matplotlib.pyplot as plt

data = np.loadtxt("mag_out.txt")
center=  [-1498.95380107  1642.24486383  4885.22294537]
radii=  [1566.19676204 2055.12161137 1869.01984246]
x_list = [row[0] for row in data]
y_list = [row[1] for row in data]
z_list = [row[2] for row in data]
v_list = []

r = (radii[0] * radii[1] * radii[2]) ** (1.0 / 3.0)
print("radius",r)
for i in range(len(x_list)):
    x_list[i] = (x_list[i]-center[0])/r#/radii[0]
    y_list[i] = (y_list[i]-center[1])/r#/radii[1]
    z_list[i] = (z_list[i]-center[2])/r#/radii[2]

    v_list.append((x_list[i]**2+y_list[i]**2+z_list[i]**2))

print(len(x_list))
value=0.1
plt.plot(x_list,y_list,'ro',alpha=value)
plt.ylabel('data')
plt.xlabel('time')
plt.show()

plt.plot(z_list,x_list,'go',alpha=value)
plt.ylabel('data')
plt.xlabel('time')
plt.show()

plt.plot(z_list,y_list,'bo',alpha=value)
plt.ylabel('data')
plt.xlabel('time')
plt.show()

plt.plot(v_list,'co',alpha=value)
plt.ylabel('vect')
plt.xlabel('time')
plt.show()

SyntaxError: invalid syntax (<ipython-input-3-e5f8bdc10143>, line 4)

In [17]:
for i in range(700):
    data = zumi.compass.get_magnet_raw()
    data[0] = (data[0]-center[0])/r
    data[1] = (data[1]-center[1])/r
    data[2] = (data[2]-center[2])/r

    rad = math.atan2(data[0],data[1])
    ang = int(math.degrees(rad))

    arg = (data[0]**2+data[1]**2)**0.5
    arg = math.atan2(arg,data[2])
    ang_xyz = int(math.degrees(arg))

    print("bearing ",ang, " xyz",ang_xyz)
    IPython.display.clear_output(wait=True) 
print("done")

done


In [ ]:
x_data_list = []
y_data_list = []
z_data_list = []
ang_list = []
for i in range(400):
    data = zumi.compass.get_magnet_raw()
    data[0] = (data[0]-center[0])/a
    data[1] = (data[1]-center[1])/b
    data[2] = (data[2]-center[2])/c
    x_data_list.append(data[0])
    y_data_list.append(data[1])
    z_data_list.append(data[2])
    rad = math.atan2(data[0],data[1])
    ang = int(math.degrees(rad))
    ang_list.append(ang)
    print("bearing ",ang)
    print(data)
    IPython.display.clear_output(wait=True) 
print("done")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ang_list,'go')
plt.ylabel('ang')
plt.xlabel('time')
plt.show()

plt.plot(x_data_list,y_data_list,'bo')
plt.ylabel('xy')
plt.xlabel('time')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

x_data_list = []
y_data_list = []
z_data_list = []
ang_list = []
try:
    for i in range(400):
        zumi.control_motors(-10,10)
        data = zumi.compass.get_magnet_raw()
        data[0] = (data[0]-center[0])/a
        data[1] = (data[1]-center[1])/b
        data[2] = (data[2]-center[2])/c
        x_data_list.append(data[0])
        y_data_list.append(data[1])
        z_data_list.append(data[2])
        rad = math.atan2(data[0],data[1])
        ang = int(math.degrees(rad))
        ang_list.append(ang)
        print(ang)
        print(data)
        IPython.display.clear_output(wait=True) 
finally:
    zumi.stop()

plt.plot(ang_list,'go')
plt.ylabel('ang')
plt.xlabel('time')
plt.show()

plt.plot(x_data_list,y_data_list,'bo')
plt.ylabel('xy')
plt.xlabel('time')
plt.show()
